In [1]:
import pytorch_lightning as pl
from lightning_denoiser import GradMatch
from data_module import DataModule
from pytorch_lightning import loggers as pl_loggers
# from pytorch_lightning.callbacks import TQDMProgressBar
import os
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from models.model_to_onnx import convert_model
from argparse import ArgumentParser
import random
import torch

/work/sc004/sc004/tc1213/miniconda3/envs/pl2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
usage: ipykernel_launcher.py [-h] [--folder_out FOLDER_OUT]
                             [--checkpoint_path CHECKPOINT_PATH]
                             [--architecture ARCHITECTURE]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"71336c65-7570-428c-af51-51c6fdb6c62c" --shell=9007 --transport="tcp" --iopub=9009


SystemExit: 2

/work/sc004/sc004/tc1213/miniconda3/envs/pl2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
os.environ['TF_ENABLE_ONEDNN_OPTS']='0'
torch.cuda.empty_cache()
parser = ArgumentParser()
parser.add_argument('--name', type=str, default='test')
parser.add_argument('--log_folder', type=str, default='logs')
parser.add_argument('--save_images', dest='save_images', action='store_true')
parser.set_defaults(save_images=False)

# MODEL args
parser = GradMatch.add_model_specific_args(parser)
# DATA args
parser = DataModule.add_data_specific_args(parser)
# OPTIM args
parser = GradMatch.add_optim_specific_args(parser)

In [ ]:

os.environ['TF_ENABLE_ONEDNN_OPTS']='0'

if __name__ == '__main__':


    # PROGRAM args
    torch.cuda.empty_cache()
    parser = ArgumentParser()
    parser.add_argument('--name', type=str, default='test')
    parser.add_argument('--log_folder', type=str, default='logs')
    parser.add_argument('--save_images', dest='save_images', action='store_true')
    parser.set_defaults(save_images=False)

    # MODEL args
    parser = GradMatch.add_model_specific_args(parser)
    # DATA args
    parser = DataModule.add_data_specific_args(parser)
    # OPTIM args
    parser = GradMatch.add_optim_specific_args(parser)

    hparams = parser.parse_args()

    refresh_rate = 144

    random.seed(hparams.seed)
    torch.manual_seed(hparams.seed)
    
    if not os.path.exists(hparams.log_folder):
        os.makedirs(hparams.log_folder, exist_ok=True)
    log_path = hparams.log_folder + '/' + hparams.name
    if not os.path.exists(log_path):
        os.makedirs(log_path, exist_ok=True)
    tb_logger = pl_loggers.TensorBoardLogger(log_path)

    model = GradMatch(hparams)
    dm = DataModule(hparams)

    if hparams.pretrained_checkpoint is not None:
        checkpoint = torch.load(hparams.pretrained_checkpoint)
        print('Loaded pretrained network from {}.'.format(hparams.pretrained_checkpoint))
        model.load_state_dict(checkpoint['state_dict'],strict=False)

    early_stop_callback = EarlyStopping(
        monitor='val/3_val_loss',
        min_delta=1e-8,
        patience=hparams.early_stopping_patiente,
        verbose=True,
        mode='min'
    )

    from pytorch_lightning.callbacks import LearningRateMonitor
    lr_monitor = LearningRateMonitor(logging_interval='epoch')

    if hparams.jacobian_loss_weight > 0:
        max_epochs = 400
        hparams.scheduler_milestones = 300
    else:
        max_epochs = 800

    # removing log_gpu_memory, if needed use DeviceStatsMonitor callback

    save_checkpoint = ModelCheckpoint(
        filename="best_{epoch}-{step}",
        monitor="val/3_val_loss",
        save_top_k=1,
        mode="min"
    )

    # save_epoch_checkpoint = ModelCheckpoint(
    #     filename="{epoch}-{step}",
    #     monitor="period",
    #     save_top_k=-50
    # )

    trainer = pl.Trainer.from_argparse_args(hparams, logger=tb_logger, gpus=-1,
                                            check_val_every_n_epoch=50,
                                            gradient_clip_val=hparams.gradient_clip_val,
                                            max_epochs=max_epochs, precision=16,
                                            callbacks=[lr_monitor,save_checkpoint], 
                                            # accelerator = 'gpu',
                                            strategy='ddp',
                                            enable_progress_bar = False)
    trainer.fit(model, dm)
    final_ckpt_pth = os.path.join(log_path,'final.ckpt')
    trainer.save_checkpoint(final_ckpt_pth)
    
    # save_pth = os.path.join('/work/sc004/sc004/tc1213/trained_networks',hparams.dataset_name, hparams.model_name) +'/seed'+str(hparams.seed)
    # save_name = hparams.model_name + '_' + hparams.dataset_name + '_' + str(hparams.sigma) + '_' + str(hparams.dr) + '_' + str(hparams.seed) + '.onnx'
    # convert_model(hparams.model_name,
    #                 os.path.join(save_pth, save_name),
    #                 final_ckpt_pth)
